In [3]:
from selenium import webdriver    # Connect chrome browser to open NYSE offical page
from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(executable_path="/Users/huzujun/Desktop/chromedriver", chrome_options=options)

In [ ]:
import time
def get_page_info(idx):
    url = f'https://www.xiaohongshu.com/discovery/item/{idx}'
    driver.get(url)
    return driver.page_source

def get_info_from_page(page):
    user_info_str = re.findall(r"<script>window.__INITIAL_SSR_STATE__[\s\S]*?{([\s\S]+?)}</script>"
                                , page)[0].strip()
    user_info_str = "{" + "{}".format(user_info_str) + "}"
    users_info = json.loads(user_info_str.replace('undefined', 'null'))
    note_info_json = users_info['NoteView']['noteInfo']

    #print(note_info_json.keys())
    note_info = {}
    note_info['desc'] = note_info_json['desc']
    note_info['hashTags'] = len(note_info_json['hashTags'])
    note_info['keywords'] = note_info_json['seoMeta']['keywords']
    note_info['user_id'] = note_info_json['user']['id']
    note_info['time'] = note_info_json['time']
    return note_info

columns_name = ['desc', 'hashTags', 'keywords', 'user_id', 'time', 'title', 'liked_count',
                'image_nums', 'nickname']
notes_info = pd.DataFrame(columns = columns_name)
keywords = ['经验', '好物', '职业', '阅读', '知识', '广州', '深圳', '珠海', '探店', '算法', '数据', 'leetcode', '学习', '程序员', '香港', '干货', '留学']
for keyword in keywords:
    for page_index in range(1, 50):
        try:
            result = wh.get_search_notes(keyword, page_index)
            items = result['result']['data']['items']
            for item in items:
                try:
                    if (item['note']['type'] == 'normal'):
                        page = get_page_info(item['note']['id'])
                        time.sleep(1)
                        note_info = get_info_from_page(page)
                        note_info['search_word'] = keyword
                        note_info['title'] = item['note']['title']
                        note_info['liked_count'] = item['note']['liked_count']
                        note_info['image_nums'] = len(item['note']['images_list'])
                        note_info['nickname'] = item['note']['user']['nickname']
                        notes_info = notes_info.append(note_info.copy(), ignore_index=True)
                except Exception as e:
                    print(e)
            print(len(notes_info))
        except Exception as e:
            print(e)
        print(notes_info.tail())
        print(keyword, page_index)